In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import os
import matplotlib.pyplot as plt
import torch

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!git clone https://github.com/LucaIanniello/MLIAProject.git

In [ ]:

!conda --version
!which conda
!conda config --show-sources
!conda env update -n base -f MLIAProject/CLAM/env.yml


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!conda env create -f MLIAProject/CLAM/env.yml
!conda install -n clam_latest -c conda-forge openslide
!conda install -n clam_latest -c conda-forge openslide-tools libopenslide0 libtiff-dev libjpeg-dev
!conda install -n clam_latest -c conda-forge future
!conda run -n clam_latest python --version
!conda run -n clam_latest pip list

!apt-get update
!apt-get install -y openslide-tools libopenslide0 libtiff-dev libjpeg-dev

In [ ]:
# the config of heatmap needs the original WSI (about 10 min to download)

# Autenticazione “di servizio” fornita da Colab
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = '1gtE_lNw4MYiarzIu4qVAo-uu6aSzxS1d'
os.makedirs('/content/ndpi_files', exist_ok=True)

# Elenca e scarica tutti i file nella cartella
file_list = drive.ListFile(
    {'q': f"'{folder_id}' in parents and trashed=false"}
).GetList()

# 1) Definisci gli array
B = [
    "M-1.ndpi",
    
    "M-104.ndpi","M-105.ndpi","M-108.ndpi",
    "M-112.ndpi","M-121.ndpi","M-24.ndpi","M-30.ndpi",
    "M-32.ndpi","M-36.ndpi"
    
]
E = [
    "M-10.ndpi",
    
    "M-100.ndpi","M-102.ndpi","M-103.ndpi",
    "M-107.ndpi","M-109.ndpi","M-11.ndpi","M-110.ndpi",
    "M-111.ndpi","M-113.ndpi"
    
]
S = [
    "M-101.ndpi",
    
    "M-114.ndpi","M-65.ndpi","M-86.ndpi","M-87.ndpi"
    
]

base_dir = '/content/ndpi_files'
for sub in ('B','E','S'):
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

for f in file_list:
    title = f['title']
    if title in B:
        dest = os.path.join(base_dir, title)
    elif title in E:
        dest = os.path.join(base_dir, title)
    elif title in S:
        dest = os.path.join(base_dir, title)
    else:
        continue

    print(f"Scarico {title} in {os.path.dirname(dest)} …")
    f.GetContentFile(dest)

In [ ]:
########## ALTERNATIVE WAY TO DOWNLOAD ###########
########## Should be a more RAM-Friendly Version #

!conda install -n clam_latest -c conda-forge gdown


import os
import gdown # Make sure gdown is installed

# Autenticazione “di servizio” fornita da Colab
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_id = '1gtE_lNw4MYiarzIu4qVAo-uu6aSzxS1d'
os.makedirs('/content/ndpi_files', exist_ok=True)

# Elenca e scarica tutti i file nella cartella
file_list = drive.ListFile(
    {'q': f"'{folder_id}' in parents and trashed=false"}
).GetList()

B = [
    "M-1.ndpi", "M-104.ndpi", "M-105.ndpi", "M-108.ndpi",
    "M-112.ndpi", "M-121.ndpi", "M-24.ndpi", "M-30.ndpi",
    "M-32.ndpi", "M-36.ndpi"
]
E = [
    "M-10.ndpi", "M-100.ndpi", "M-102.ndpi", "M-103.ndpi",
    "M-107.ndpi", "M-109.ndpi", "M-11.ndpi", "M-110.ndpi",
    "M-111.ndpi", "M-113.ndpi"
]
S = [
    "M-101.ndpi", "M-114.ndpi", "M-65.ndpi", "M-86.ndpi", "M-87.ndpi"
]

base_dir = '/content/ndpi_files'
for sub in ('B','E','S'):
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

for f in file_list:
    title = f['title']
    file_id = f['id']

    if title in B or title in E or title in S:
        dest_dir = base_dir
        if title in B:
            dest_dir = os.path.join(base_dir, 'B')
        elif title in E:
            dest_dir = os.path.join(base_dir, 'E')
        elif title in S:
            dest_dir = os.path.join(base_dir, 'S')

        dest_path = os.path.join(dest_dir, title)

        # Check if the file already exists
        if os.path.exists(dest_path):
            print(f"File {title} already exists. Skipping download.")
        else:
            print(f"Scarico {title} ({file_id}) in {dest_path} …")
            try:
                gdown.download(id=file_id, output=dest_path, quiet=False)
            except Exception as e:
                print(f"Errore durante il download di {title}: {e}")
    else:
        continue

In [ ]:
# the config for the heatmap needs this csv file

import csv
import os

# Combine arrays with their labels
data = []
for slide in B:
    data.append([slide.replace(".ndpi", ""), "B"])
for slide in E:
    data.append([slide.replace(".ndpi", ""), "E"])
for slide in S:
    data.append([slide.replace(".ndpi", ""), "S"])

# Specify the directory for the output file
output_directory = r"MLIAProject/CLAM/heatmaps/process_lists/"

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Specify the full path for the output file
output_file = os.path.join("heatmap_dataset.csv") # output_directory, 

# Write to CSV file
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["slide_id", "label"])  # Write header
    writer.writerows(data)  # Write data

print(f"CSV file '{output_file}' created successfully.")

In [ ]:
######## ONE IMAGE PER CLASS ########

import csv
import os


# Select one image from each list
selected_images = []
if B:
    selected_images.append([B[1].replace(".ndpi", ""), "B"])
if E:
    selected_images.append([E[6].replace(".ndpi", ""), "E"])
if S:
    selected_images.append([S[0].replace(".ndpi", ""), "S"])

# Specify the output directory and filename for the new CSV
output_directory = r"MLIAProject/CLAM/heatmaps/process_lists/"
output_file = os.path.join("single_image_dataset.csv")

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Write the data to the CSV file
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["slide_id", "label"])  # Write header
    writer.writerows(selected_images)  # Write data

print(f"Test CSV file '{output_file}' created successfully with {len(selected_images)} entries.")

In [ ]:
# the config of heatmap needs a checkpoint to initialize the model
# (in this case results/MLIA_CLAM_50_s1/s_0_checkpoint.pt)

!unzip MLIAProject/train_eval_results/CLAM_results/result_classifier.zip

In [ ]:
!MPLBACKEND=Agg CUDA_VISIBLE_DEVICES=0 conda run -n clam_latest python MLIAProject/CLAM/create_heatmaps.py --config MLIAProject/CLAM/heatmaps/configs/MLIAProject_config.yaml
